In [9]:
import os
import pandas as pd
import numpy as np
import sys
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.io import read_image
import torchio as tio
import matplotlib.pyplot as plt


sys.path.insert(0, '../../scripts/')

from helpers import miscellaneous as m
from helpers import preprocessing2d as prep
from helpers import plotters
from data_loader import MRIDatasetSlice
from ml_models import get_model
from loss_functions import get_optimizer, get_criterion
import numpy as np
from torchcam.methods import SmoothGradCAMpp
from torchvision.transforms.functional import to_pil_image
from torchcam.utils import overlay_mask



CONFIG = m.get_config()
MODEL_PATH = '../../models/'

In [3]:
df  = pd.read_csv('../../data/annotations/test_labels.csv')
df.head()

,filename,Group,Subject
0,data/raw/flattened\ADNI_141_S_1052_MR_MPR__Gra...,MCI,141_S_1052
1,data/raw/flattened\ADNI_141_S_1052_MR_MPR__Gra...,MCI,141_S_1052
2,data/raw/flattened\ADNI_141_S_1052_MR_MPR__Gra...,MCI,141_S_1052
3,data/raw/flattened\ADNI_141_S_0851_MR_MPR__Gra...,MCI,141_S_0851
4,data/raw/flattened\ADNI_141_S_0851_MR_MPR-R__G...,MCI,141_S_0851


### Load Model

In [4]:
MODEL_NAME = 'ResNet18_CV_LR0.0014926327291968032_CRFocal Loss_TRMonai_Blur_OPAdamW_BS16_EP30__ID_d066a171-10e9-4d9b-8999-c93a0a982283.pth'
model = get_model("ResNet18")

TRANSFORMER_NAME = "Monai_Blur"
BATCH_SIZE = 16
EPOCHS = 20
CRITERION = "Focal Loss"
OPTIMIZER = "AdamW"

criterion = get_criterion({"CRITERION":CRITERION})
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
transformer = prep.get_transformer(TRANSFORMER_NAME)[0]

state_dict = torch.load(MODEL_PATH + MODEL_NAME)

#state_dict
model.load_state_dict(state_dict['model_state_dict'])

<All keys matched successfully>

In [5]:
test_data = MRIDatasetSlice('../../data/annotations/test_labels.csv','../../' + CONFIG['FLATTENED_DATA_DIR'], transformer,  'Center', 3)

test_dataloader = DataLoader(test_data, batch_size=1, shuffle=False)

In [6]:
df_testset = pd.read_csv('../../data/annotations/test_labels.csv')
df_testset.head()

,filename,Group,Subject
0,data/raw/flattened\ADNI_141_S_1052_MR_MPR__Gra...,MCI,141_S_1052
1,data/raw/flattened\ADNI_141_S_1052_MR_MPR__Gra...,MCI,141_S_1052
2,data/raw/flattened\ADNI_141_S_1052_MR_MPR__Gra...,MCI,141_S_1052
3,data/raw/flattened\ADNI_141_S_0851_MR_MPR__Gra...,MCI,141_S_0851
4,data/raw/flattened\ADNI_141_S_0851_MR_MPR-R__G...,MCI,141_S_0851


In [18]:
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50

def GradCam(img_path):
    image = tio.ScalarImage(img_path).data.float().to(device)
    image = image.permute((0,3,1,2)).squeeze(dim=3)
    
    grad_cam(model, [model.fc], image, transformer_name='Monai_Blur')
    
def grad_cam(model, target_layer, image, transformer_name=None):
    if transformer_name:
        transformer, _  = prep.get_transformer(transformer_name)

        input_tensor = transformer(image.squeeze(0))
    elif transformer_name is None:
        input_tensor = image

    model = model.eval()

    cam_extractor = SmoothGradCAMpp(model, target_layer=target_layer)

    input_tensor = input_tensor.unsqueeze(dim=0)

    out = model(input_tensor)

    activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)

    result = overlay_mask(to_pil_image(image), to_pil_image(activation_map[0].squeeze(0), mode='F'), alpha=0.5)
    plt.figure(figsize=(8,8))
    plt.imshow(result); plt.axis('off'); plt.tight_layout(); plt.show()
    
img_path = os.path.join('../../' + CONFIG['SLICED_DATA_DIR'] + 'center/3/' , df_testset.filename[0].replace('data/raw/flattened\\', ''))
GradCam(img_path)

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [ ]:
import matplotlib.pyplot as plt
tensor_to_image = transforms.Compose([
    transforms.ToPILImage(),
])

for data in test_dataloader:
    plt.imshow(data[0].permute(2, 3, 0, 1).squeeze(2))
    break

In [ ]:
model